In [17]:
from qdrant_client import QdrantClient, models
import requests
from fastembed import TextEmbedding
import json
import random
from openai import OpenAI

In [ ]:
qdr_client = QdrantClient("http://localhost:6333")
openai_client = OpenAI()
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [19]:
documents = []

for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [20]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [21]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [22]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"
collection_name = "zoomcamp-rag"

In [23]:
qdr_client.delete_collection(collection_name)

True

In [24]:
# Create the collection with specified vector parameters
qdr_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [25]:
qdr_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [26]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [27]:
qdr_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [30]:
def vector_search(query,course='data-engineering-zoomcamp', limit=3):
    print('vector_search is used')
    
    query_points = qdr_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit,
        with_payload=True
    )
    
    results = []
    for point in query_points.points:
        results.append(point.payload)
    return results

In [33]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [34]:
rag('how do I run kafka?')

vector_search is used


'To run Kafka, you can follow these steps based on the provided context:\n\n1. **Run the Producer:** In the project directory, execute the following command in your terminal to run the Kafka producer. Make sure to replace `<jar_name>` with the actual name of your jar file:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n2. **Run the Consumer:** Similarly, if you want to run the Kafka consumer, you would execute:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonConsumer.java\n   ```\n\n3. **Ensure Kafka Broker is Running:** Before running the producer or consumer, confirm that your Kafka broker is up and running. You can check this by executing `docker ps` in your terminal. If the Kafka broker is not running, navigate to the folder containing your docker compose YAML file and run:\n   ```\n   docker compose up -d\n   ``` \n\nPlease make sure to check your Kafka configurati

In [59]:
## Homework

In [85]:
query_embd = ['I just discovered the course. Can I join now?']

In [89]:
model = TextEmbedding(model_name=model_handle)
query_array_embd = model.embed(query_embd)
query_embeddings = list(query_array_embd)

In [90]:
min(list(query_embeddings[0]))

-0.11726373885183883

In [78]:
import numpy as np

In [91]:
np.linalg.norm(query_embeddings)

1.0

In [92]:
np.dot(query_embeddings[0],query_embeddings[0])

1.0000000000000002

In [84]:
doc = 'Can I still join the course after the start date?'

In [93]:
doc_array_embd = model.embed(doc)
doc_embeddings = list(doc_array_embd)

In [94]:
np.dot(query_embeddings[0],doc_embeddings[0])

0.9008528895674548

In [96]:
query_embeddings[0].dot(doc_embeddings[0])

0.9008528895674548

In [95]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [97]:
text_embeddings = []

In [98]:
for doc in documents:
    array_embd = model.embed(doc['text'])
    embeddings = list(array_embd)
    text_embeddings.extend(embeddings)

In [101]:
text_embeddings

[array([-2.49525006e-02, -3.96454034e-02, -4.37672429e-03,  2.95830380e-02,
        -1.20300744e-02, -6.93249608e-03, -7.65465072e-05,  6.15121401e-02,
        -2.85105823e-02, -6.70960944e-02,  6.53191995e-03, -5.09575722e-02,
        -5.01244174e-02, -5.83276165e-02,  9.54578951e-02, -5.41039925e-02,
        -3.20385860e-02, -4.69560323e-02, -6.16868024e-02, -9.03934389e-02,
        -3.35349282e-02, -2.90533971e-02,  4.44972092e-02, -7.00045540e-02,
         4.27595264e-02, -7.01989178e-02, -8.68037276e-02,  8.14205881e-02,
         7.21879907e-02,  1.89472918e-02, -5.95576718e-02,  1.09493119e-02,
        -1.80511540e-02,  4.60817619e-02, -2.97588624e-02,  3.45792057e-02,
         4.05221611e-02,  1.75949846e-02, -2.77041533e-02, -1.67211727e-02,
        -5.02859986e-02,  1.34971538e-02,  9.17410131e-02,  8.32858616e-03,
        -7.49552827e-02,  2.35092282e-03, -4.66536576e-02,  1.29560582e-02,
         2.57738903e-02,  5.87156475e-03, -4.75062527e-02,  1.41338343e-02,
        -1.0

In [103]:
cosine_similarity = []
for vector in text_embeddings:
    cosine_similarity.append(query_embeddings[0].dot(vector))

In [104]:
cosine_similarity

[0.7629684696540237,
 0.818237815004289,
 0.8085397398734189,
 0.7133079015686243,
 0.7304499234333599]

In [105]:
max(cosine_similarity)

0.818237815004289

In [106]:
full_text_embeddings = []
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    array_embd = model.embed(full_text)
    embeddings = list(array_embd)
    full_text_embeddings.extend(embeddings)

In [107]:
full_cosine_similarity = []
for vector in full_text_embeddings:
    full_cosine_similarity.append(query_embeddings[0].dot(vector))

In [109]:
max(full_cosine_similarity)

0.8514543236908066

In [110]:
full_cosine_similarity

[0.8514543236908066,
 0.8436594159113069,
 0.8408287048502558,
 0.7755157969663908,
 0.8086007795043937]

In [116]:
dim_set = set()
for model in TextEmbedding.list_supported_models():
    if model["dim"]:
        dim_set.add(model["dim"])

In [119]:
min(dim_set)

384

In [118]:
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [120]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [121]:
EMBEDDING_DIMENSIONALITY = 384
model_handle = "BAAI/bge-small-en"
collection_name = "llm-zoomcamp-homework"

In [125]:
qdr_client.delete_collection(collection_name)

True

In [126]:
# Create the collection with specified vector parameters
qdr_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [127]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [128]:
qdr_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [135]:
def vector_search(query,course='machine-learning-zoomcamp', limit=1):
    print('vector_search is used')
    
    query_points = qdr_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit,
        with_payload=True
    )
    
    results = []
    for point in query_points.points:
        results.append(point)
    return results

In [136]:
vector_search('I just discovered the course. Can I join now?')

vector_search is used


[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)]